<a href="https://www.kaggle.com/code/abbas829/line-charts-guideline?scriptVersionId=297666747" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Line Charts — clean, reproducible, error-free


In [ ]:
# 📦 Import Essential Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# 🎨 Set Visualization Styles
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Libraries imported successfully!")
print("📊 Ready to create amazing Line Charts!")

In [ ]:
# --- Data generation (synthetic, reproducible) ---

def generate_flights(start_year=1949, end_year=1960, seed=42):
    """Return a DataFrame with monthly passenger counts (synthetic)."""
    np.random.seed(seed)
    months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    rows = []
    for i, year in enumerate(range(start_year, end_year + 1)):
        for j, month in enumerate(months):
            base = 112 + (i * 30)                    # upward trend by year
            seasonal = 40 * np.sin(2 * np.pi * (j - 2) / 12)  # seasonality
            val = int(base + seasonal + np.random.randint(-15, 15))
            rows.append([year, month, max(50, val)])
    df = pd.DataFrame(rows, columns=['year','month','passengers'])
    df['month'] = pd.Categorical(df['month'], categories=months, ordered=True)
    # build a valid datetime from year + abbreviated month name
    df['date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str) + '-01', format='%Y-%b-%d')
    return df


def generate_stock(start='2023-01-01', end='2023-12-31', seed=42):
    """Return a daily synthetic stock price + volume DataFrame."""
    np.random.seed(seed)
    dates = pd.date_range(start, end, freq='D')
    price = 100 + np.cumsum(np.random.randn(len(dates)) * 2)
    volume = np.random.randint(1_000_000, 5_000_000, size=len(dates))
    return pd.DataFrame({'date': dates, 'price': price, 'volume': volume, 'company': ['TechCorp'] * len(dates)})

# Create datasets used in the notebook
flights = generate_flights()
stock_data = generate_stock()

print(f"Flights: {flights.shape} — example:\n", flights.head(6))
print(f"\nStock data: {stock_data.shape} — example:\n", stock_data.head(3))

---

## 4. Matplotlib Line Charts

**Matplotlib** is the grandfather of Python visualization libraries. It provides fine-grained control over every aspect of your chart.

### 4.1 Basic Line Chart

In [ ]:
# 📈 Basic Line Chart with Matplotlib
plt.figure(figsize=(12, 6))

# Aggregate flights data by year
yearly_passengers = flights.groupby('year')['passengers'].sum()

plt.plot(yearly_passengers.index, yearly_passengers.values, 
         marker='o', linewidth=2.5, markersize=8, color='#2E86AB')

plt.title('Total Air Passengers by Year (1949-1960)', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Total Passengers', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("💡 Insight: Clear upward trend in air travel popularity!")

## 💡 Analysis & Interpretation

**What this chart (Total Passengers by Year) shows:**
- **Overall trend:** steady, accelerating growth in total annual passengers between 1949 and 1960.
- **Magnitude:** passenger totals roughly quadruple across the period — clearly non-linear growth.

**How to read it (actionable):**
- Use the yearly aggregate to track long-term growth and high-level season-independent changes.
- Drill down into monthly plots (see below) to inspect seasonality and short-term peaks.

**Key takeaway:** the airline market expands rapidly over this period — planning should account for increasing capacity and seasonal peaks.

### 4.2 Styled Line Chart with Annotations

In [ ]:
# 🎨 Advanced Styled Line Chart
fig, ax = plt.subplots(figsize=(14, 7))

# Use existing 'date' column if present; otherwise create it reliably
if 'date' not in flights.columns:
    flights['date'] = pd.to_datetime(flights['year'].astype(str) + '-' + flights['month'].astype(str) + '-01')

monthly_data = flights.sort_values('date')

# Plot with gradient effect
ax.plot(monthly_data['date'], monthly_data['passengers'], 
        linewidth=3, color='#A23B72', alpha=0.8)

# Fill area under curve
ax.fill_between(monthly_data['date'], monthly_data['passengers'], 
                alpha=0.3, color='#F18F01')

# Add peak annotation
max_idx = monthly_data['passengers'].idxmax()
max_row = monthly_data.loc[max_idx]
ax.annotate(f'Peak: {max_row["passengers"]} passengers\n{max_row["date"].strftime("%B %Y")}', 
            xy=(max_row['date'], max_row['passengers']), 
            xytext=(max_row['date'], max_row['passengers'] + 50),
            arrowprops=dict(arrowstyle='->', color='red', lw=2),
            fontsize=11, ha='center', bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.7))

ax.set_title('Airline Passengers Over Time with Trend Analysis', fontsize=16, fontweight='bold')
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Number of Passengers', fontsize=12)
ax.grid(True, alpha=0.3, linestyle='--')

# Rotate x-axis labels
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 💡 Understanding the Seasonality

**Visual Insights:**
- **Cyclical Patterns:** Unlike the yearly aggregate, this detailed view shows strong seasonality. Every year follows a similar pattern of ups and downs.
- **Peak Travel:** The **Peaks** occur consistently in the summer months (July & August), indicating holiday travel demand.
- **Troughs:** The lowest points are typically in the fall/winter months, reflecting reduced leisure travel.
- **Annotation:** The highlighted peak shows the maximum passenger value in the dataset, effectively drawing attention to the most significant data point.


### 4.3 Multiple Line Chart

In [ ]:
# 📊 Multiple Lines Comparison
fig, ax = plt.subplots(figsize=(14, 7))

# Pivot data to get years as columns
pivot_flights = flights.pivot(index='month', columns='year', values='passengers')

# Plot multiple years
colors = plt.cm.viridis(np.linspace(0, 1, len(pivot_flights.columns)))
for i, year in enumerate(pivot_flights.columns):
    ax.plot(pivot_flights.index, pivot_flights[year], 
            marker='o', linewidth=2, label=f'{year}', color=colors[i], markersize=6)

ax.set_title('Monthly Passengers Comparison Across Years', fontsize=16, fontweight='bold')
ax.set_xlabel('Month', fontsize=12)
ax.set_ylabel('Passengers', fontsize=12)
ax.legend(title='Year', bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("💡 Insight: Summer months (July-August) consistently show peak travel!")

**Explanation — Monthly comparison across years:**

- Each line shows the monthly passenger pattern for a single year; parallel shapes across years indicate consistent seasonality.
- Notice the **higher peaks in summer months** (July/Aug) and a gradual upward shift in baseline across successive years.
- Use this view to compare seasonal amplitude year-over-year and to spot outlier years.

---

## 5. Seaborn Line Charts

**Seaborn** is built on top of Matplotlib and provides a high-level interface for drawing attractive statistical graphics.

### 5.1 Basic Seaborn Line Plot

In [ ]:
# 🌊 Seaborn Line Plot
plt.figure(figsize=(12, 6))

# Create line plot with confidence interval
sns.lineplot(data=flights, x='year', y='passengers', 
             marker='o', linewidth=2.5, markersize=8, color='#E63946')

plt.title('Average Passengers per Year (Seaborn)', fontsize=16, fontweight='bold')
plt.xlabel('Year', fontsize=12)
plt.ylabel('Passengers', fontsize=12)
plt.tight_layout()
plt.show()

### 5.2 Grouped Line Chart with Seaborn

In [ ]:
# 🎨 Grouped Line Chart
plt.figure(figsize=(14, 7))

# Line plot with hue for different years
sns.lineplot(data=flights, x='month', y='passengers', hue='year', 
             palette='tab10', marker='o', linewidth=2, markersize=8)

plt.title('Monthly Passenger Trends by Year', fontsize=16, fontweight='bold')
plt.xlabel('Month', fontsize=12)
plt.ylabel('Number of Passengers', fontsize=12)
plt.legend(title='Year', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 5.3 Advanced Seaborn with FacetGrid

In [ ]:
# 🔥 FacetGrid for Multiple Subplots
g = sns.FacetGrid(flights, col='year', col_wrap=4, height=3, aspect=1.2)
g.map(sns.lineplot, 'month', 'passengers', marker='o', color='#2A9D8F')
g.set_axis_labels('Month', 'Passengers')
g.set_titles(col_template='{col_name}')
g.fig.suptitle('Yearly Passenger Trends (Individual Views)', y=1.02, fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

---

## 6. Plotly Interactive Line Charts

**Plotly** creates interactive, web-based visualizations that are perfect for dashboards and presentations.

### 6.1 Basic Interactive Line Chart

In [ ]:
# 🚀 Plotly Express Line Chart
fig = px.line(flights, x='month', y='passengers', color='year', 
              title='Interactive: Monthly Passengers by Year',
              labels={'passengers': 'Number of Passengers', 'month': 'Month'},
              template='plotly_white')

fig.update_traces(mode='lines+markers', marker=dict(size=8))
fig.update_layout(
    title_font_size=20,
    xaxis_title_font_size=14,
    yaxis_title_font_size=14,
    legend_title_text='Year',
    hovermode='x unified',
    height=600
)



**Explanation — Interactive Plotly chart:**

- Hover to see exact monthly passenger values for each year.
- The interactive legend lets you toggle individual years on/off — useful to de-clutter the visualization.
- Use this chart for exploratory analysis and presentations where viewers can interact with the data.

### 6.2 Advanced Interactive Chart with Range Slider

In [ ]:
# 📊 Time Series with Range Slider
fig = go.Figure()

# Add main line
fig.add_trace(go.Scatter(
    x=monthly_data['date'],
    y=monthly_data['passengers'],
    mode='lines',
    name='Passengers',
    line=dict(color='#FF6B6B', width=3),
    fill='tozeroy',
    fillcolor='rgba(255, 107, 107, 0.2)'
))

# Add moving average
monthly_data['ma_6'] = monthly_data['passengers'].rolling(window=6).mean()
fig.add_trace(go.Scatter(
    x=monthly_data['date'],
    y=monthly_data['ma_6'],
    mode='lines',
    name='6-Month Moving Average',
    line=dict(color='#4ECDC4', width=3, dash='dash')
))

# Update layout with range slider
fig.update_layout(
    title='Airline Passengers with Moving Average & Range Slider',
    xaxis_title='Date',
    yaxis_title='Passengers',
    xaxis_rangeslider_visible=True,
    template='plotly_white',
    height=700,
    hovermode='x unified'
)


**Explanation — Range slider & moving average:**

- The range slider allows zooming into any sub-period without changing the data.
- The 6-month moving average smooths short-term volatility and reveals the underlying trend.
- Use the moving-average trace to highlight structural shifts while keeping the raw series visible.

### 6.3 Multi-Axis Interactive Chart

In [ ]:
# 🎯 Dual Axis Chart
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Sample stock data aggregation
stock_monthly = stock_data.set_index('date').resample('M').agg({
    'price': 'mean',
    'volume': 'sum'
}).reset_index()

fig.add_trace(
    go.Scatter(x=stock_monthly['date'], y=stock_monthly['price'], 
               name="Stock Price", line=dict(color='#E63946', width=3)),
    secondary_y=False,
)

fig.add_trace(
    go.Bar(x=stock_monthly['date'], y=stock_monthly['volume'], 
           name="Volume", marker_color='rgba(69, 123, 157, 0.6)'),
    secondary_y=True,
)

fig.update_layout(
    title_text="Stock Price vs Trading Volume Analysis",
    template='plotly_white',
    height=600,
    hovermode='x unified'
)

fig.update_yaxes(title_text="Stock Price ($)", secondary_y=False)
fig.update_yaxes(title_text="Volume", secondary_y=True)



**Explanation — Dual-axis chart (Price vs Volume):**

- Left axis (line) shows average monthly stock price; right axis (bars) shows monthly trading volume.
- Dual axes are appropriate here because price and volume use different units — interpret correlation carefully.
- Use hover and unified x-axis to compare simultaneous movements (price spikes vs volume surges).

---

## 7. Advanced Techniques

### 7.1 Step Chart (for Discrete Changes)

In [ ]:
# 📊 Step Chart
fig, ax = plt.subplots(figsize=(12, 6))

# Create step data
quarters = ['Q1', 'Q2', 'Q3', 'Q4']
revenue_2022 = [100, 120, 115, 140]
revenue_2023 = [110, 135, 130, 160]

ax.step(quarters, revenue_2022, where='mid', label='2022', linewidth=3, marker='o', color='#457B9D')
ax.step(quarters, revenue_2023, where='mid', label='2023', linewidth=3, marker='s', color='#E63946')

ax.set_title('Quarterly Revenue Comparison (Step Chart)', fontsize=16, fontweight='bold')
ax.set_xlabel('Quarter', fontsize=12)
ax.set_ylabel('Revenue (Million $)', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("💡 Step charts are best for showing discrete changes at specific points!")

### 7.2 Sparklines (Mini Line Charts)

In [ ]:
# ✨ Sparklines
fig, axes = plt.subplots(3, 1, figsize=(12, 8))

# Generate sample data for sparklines
np.random.seed(123)
metrics = ['Revenue', 'Users', 'Conversion Rate']
colors = ['#2A9D8F', '#E9C46A', '#F4A261']

for i, (metric, color) in enumerate(zip(metrics, colors)):
    data = np.cumsum(np.random.randn(50)) + 100
    axes[i].plot(data, color=color, linewidth=2)
    axes[i].fill_between(range(len(data)), data, alpha=0.3, color=color)
    axes[i].set_ylabel(metric, fontsize=11, fontweight='bold')
    axes[i].set_xticks([])
    axes[i].set_yticks([])
    axes[i].spines['top'].set_visible(False)
    axes[i].spines['right'].set_visible(False)
    axes[i].spines['bottom'].set_visible(False)
    axes[i].spines['left'].set_visible(False)
    
    # Add end value
    axes[i].text(len(data)-1, data[-1], f'{data[-1]:.1f}', 
                va='center', ha='left', fontsize=10, fontweight='bold')

plt.suptitle('Performance Metrics Sparklines', fontsize=16, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

print("✨ Sparklines provide at-a-glance trend information!")

### 7.3 Area Chart with Gradient

In [ ]:
# 🌈 Gradient Area Chart
fig, ax = plt.subplots(figsize=(14, 7))

x = np.arange(len(monthly_data))
y = monthly_data['passengers'].values

# Create gradient fill
gradient = np.linspace(0, 1, len(x))
for i in range(len(x)-1):
    ax.fill_between([x[i], x[i+1]], [y[i], y[i+1]], alpha=0.6, 
                    color=plt.cm.plasma(gradient[i]))

ax.plot(x, y, color='white', linewidth=3)
ax.set_xticks(x[::12])
ax.set_xticklabels(monthly_data['date'].dt.year.unique())
ax.set_title('Passenger Trends with Gradient Fill', fontsize=16, fontweight='bold')
ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Passengers', fontsize=12)
plt.tight_layout()
plt.show()

## 8. Best Practices and Common Mistakes

### ✅ Best Practices

- Start the y-axis at zero when comparing magnitudes (unless a zoom is justified).
- Use consistent time intervals on the x-axis.
- Limit the number of lines to 3–5 for readability; use small multiples for many series.
- Add annotations for key events and direct labels where possible.
- When using dual axes, clearly document units to avoid misinterpretation.

### ❌ Common Mistakes

- Overcrowding (too many lines) — use filters or facets.
- Inconsistent or truncated scales that mislead comparisons.
- Using 3D or decorative effects that distort perception.

**Pro tip:** prefer simple, well-labeled charts — clarity beats cleverness.

---

## 9. Real-World Case Study

### Scenario: Analyzing Airline Performance

Let's create a comprehensive dashboard-style analysis using all techniques learned.

In [ ]:
# 🎯 Comprehensive Dashboard
fig = plt.figure(figsize=(16, 12))

# Create grid
gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)

# 1. Main Trend Line
ax1 = fig.add_subplot(gs[0, :])
ax1.plot(monthly_data['date'], monthly_data['passengers'], linewidth=3, color='#2E86AB')
ax1.fill_between(monthly_data['date'], monthly_data['passengers'], alpha=0.3, color='#2E86AB')
ax1.set_title('Overall Passenger Trend (1949-1960)', fontsize=14, fontweight='bold')
ax1.set_ylabel('Passengers')

# 2. Year-over-Year Growth
ax2 = fig.add_subplot(gs[1, 0])
yearly = flights.groupby('year')['passengers'].sum()
growth = yearly.pct_change() * 100
colors = ['green' if x > 0 else 'red' for x in growth.dropna()]
ax2.bar(growth.dropna().index, growth.dropna().values, color=colors, alpha=0.7)
ax2.set_title('Year-over-Year Growth %', fontsize=12, fontweight='bold')
ax2.set_ylabel('Growth %')
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)

# 3. Seasonal Pattern
ax3 = fig.add_subplot(gs[1, 1])
seasonal = flights.groupby('month')['passengers'].mean()
ax3.plot(seasonal.index, seasonal.values, marker='o', linewidth=2, color='#A23B72', markersize=8)
ax3.set_title('Average Seasonal Pattern', fontsize=12, fontweight='bold')
ax3.set_ylabel('Avg Passengers')

# 4. Monthly Heatmap
ax4 = fig.add_subplot(gs[2, :])
pivot = flights.pivot(index='year', columns='month', values='passengers')
sns.heatmap(pivot, annot=True, fmt='d', cmap='YlOrRd', ax=ax4, cbar_kws={'label': 'Passengers'})
ax4.set_title('Passenger Volume Heatmap by Year and Month', fontsize=12, fontweight='bold')

plt.suptitle('✈️ Airline Performance Dashboard', fontsize=18, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

print("🎯 Key Insights:")
print(f"   • Peak Year: {yearly.idxmax()} with {yearly.max():,} passengers")
print(f"   • Highest Growth: {growth.idxmax()} ({growth.max():.1f}%)")
print(f"   • Peak Season: {seasonal.idxmax()} (avg {seasonal.max():.0f} passengers)")

**Dashboard explanations & key insights:**

- **Overall Trend:** visible in the main chart — sustained yearly growth.
- **YoY Growth:** bar chart highlights years with stronger/weaker growth rates (useful for capacity planning).
- **Seasonality (average):** confirms summer peaks and winter troughs — important for staffing and pricing.

Actionable summary:
- Increase capacity planning and staffing ahead of summer months.
- Monitor years with abnormal YoY growth for operational/market signals.

---

## 10. Conclusion

### 🎓 What We've Learned:

1. **Fundamentals:** Line charts are essential for time-series and trend analysis
2. **Libraries:**
   - **Matplotlib:** Maximum customization, publication-ready static charts
   - **Seaborn:** Statistical visualization with minimal code
   - **Plotly:** Interactive web-based visualizations
3. **Best Practices:** Proper scaling, limiting lines, clear labeling
4. **Advanced Techniques:** Sparklines, step charts, area fills, dual axes

### 🚀 Next Steps:

- Experiment with your own datasets
- Explore 3D line plots for multi-dimensional data
- Combine line charts with other chart types
- Create animated line charts for time evolution

### 📚 Additional Resources:

- [Matplotlib Documentation](https://matplotlib.org/)
- [Seaborn Tutorial](https://seaborn.pydata.org/tutorial.html)
- [Plotly Python Guide](https://plotly.com/python/)

---


**Auther:** Tassawar Abbas
**Email:** abbas829@gmail.com